In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from os import listdir
import json
import networkx as nx

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('./drive', force_remount=True)

path_prefix = './drive/My Drive/CS48002-NetworkScience/'

Mounted at ./drive


In [6]:
df = pd.read_csv(path_prefix+'network_dataset.csv', index_col=0)

In [8]:
df.sample(10)

,App Id,All Permissions,App Name,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
804,com.ea.games.r3_row,"[{'permission': 'add or remove accounts', 'typ...",Real Racing 3,Racing,4.3,6670428.0,"100,000,000+",100000000.0,290620964,True,...,http://ea.com/android,help@eamobile.com,NaN,"Jun 07, 2021",Everyone,http://privacy.ea.com/en,False,True,True,2021-06-16 09:57:38
177,com.mobisystems.fileman,"[{'permission': 'view Wi-Fi connections', 'typ...",File Commander - File Manager & Free Cloud,Business,4.2,889662.0,"100,000,000+",100000000.0,162147586,True,...,http://www.mobisystems.com,support-fc@mobisystems.com,"Nov 6, 2012","May 27, 2021",Everyone,http://www.mobisystems.com/mobile/privacy-poli...,True,True,False,2021-06-16 04:37:48
902,com.huawei.himovie.overseas,"[{'permission': 'retrieve running apps', 'type...",HUAWEI Video,Entertainment,3.9,70580.0,"100,000,000+",100000000.0,450787692,True,...,https://huaweimobileservices.com/,mobilevideo@huawei.com,"Mar 18, 2020","Mar 25, 2020",Teen,https://consumer.huawei.com/en/legal/privacy-p...,False,False,False,2021-06-16 11:02:46
734,com.sonyericsson.extras.liveware,"[{'permission': 'find accounts on the device',...",Smart Connect,Tools,3.8,195990.0,"100,000,000+",100000000.0,238046010,True,...,http://www.sonymobile.com,apps@sonymobile.com,"Oct 13, 2010","Apr 01, 2021",Everyone,http://www.sonymobile.com/global-en/legal/priv...,False,False,False,2021-06-16 07:47:07
109,com.fgol.HungrySharkEvolution,"[{'permission': 'view Wi-Fi connections', 'typ...",Hungry Shark Evolution - Offline survival game,Arcade,4.4,7213140.0,"100,000,000+",100000000.0,390613416,True,...,https://www.ubisoft.com/,android.support@ubisoft.com,"Feb 22, 2013","Jun 14, 2021",Teen,http://r-mob.ubi.com/?a=PrivacyPolicyUS,True,True,False,2021-06-16 01:46:01
75,com.jb.emoji.gokeyboard,"[{'permission': 'retrieve running apps', 'type...","GO Keyboard - Cute Emojis, Themes and GIFs",Tools,4.5,4765658.0,"100,000,000+",100000000.0,229615245,True,...,http://www.gomo.com,gokeyboarddevteam@gomo.com,"Jan 13, 2015","May 13, 2021",Everyone,http://d2prafqgniatg5.cloudfront.net/GOMO/GOKe...,True,True,False,2021-06-16 03:06:47
894,com.outfit7.tomslovelettersfree,[{'permission': 'read the contents of your USB...,Tom's Love Letters,Entertainment,3.8,758080.0,"100,000,000+",100000000.0,100456954,True,...,http://outfit7.com/contact/android/,support@outfit7.com,"Feb 3, 2012","Apr 30, 2018",Everyone,http://outfit7.com/privacy/,True,False,False,2021-06-16 01:34:30
975,com.giantssoftware.fs14,[{'permission': 'modify or delete the contents...,Farming Simulator 14,Simulation,4.2,916524.0,"100,000,000+",100000000.0,107348637,True,...,http://www.farming-simulator.com/contact.php,support@giants-software.com,"Nov 17, 2013","Jul 17, 2017",Everyone,http://www.farming-simulator.com/privacyPolicy...,False,True,False,2021-06-16 11:15:04
65,com.qihoo.security,"[{'permission': 'retrieve running apps', 'type...","Safe Security - Antivirus, Booster, Phone Cle...",Tools,4.6,16783533.0,"100,000,000+",100000000.0,464387773,True,...,http://www.safesecurityapps.com,mobimagicdevelop@gmail.com,"Jun 11, 2013","Jun 18, 2020",Everyone,http://www.safesecurityapps.com/en-us/about/pr...,True,True,False,2021-06-15 21:56:20
779,loseweight.weightloss.workout.fitness,"[{'permission': 'find accounts on the device',...",Lose Weight at Home - Home Workout in 30 Days,Health & Fitness,4.8,758409.0,"50,000,000+",50000000.0,79605574,True,...,http://simpledesign.ltd,abishkking@gmail.com,"Jul 31, 2017","Apr 30, 2021",Everyone,http://simpledesign.ltd/privacypolicy.html,True,True,False,2021-06-16 11:24:07


In [4]:
def permissionSimilarity(id1, id2):
  row1=df[df["App Id"]==id1]
  row2=df[df["App Id"]==id2]

  perms1=list(row1["All Permissions"])[0]
  perms2=list(row2["All Permissions"])[0]

  perms1 = list(eval(perms1))
  perms2 = list(eval(perms2))

  common_perms = set()

  for per1 in perms1:
    for per2 in perms2:
      if dict(per1)["permission"]==dict(per2)["permission"]:
        common_perms.add(dict(per1)["permission"])
  
  if len(perms1)+ len(perms2) == 0:
    return 1
  
  return len(common_perms) / (((len(perms1)+len(perms2))/2))

In [ ]:
import csv
from google.colab import files

edgeList = []

for i, row1 in df.iterrows():
  for j,row2 in df.iterrows():
    if row1["App Id"] != row2["App Id"] and j > i:
      edgeWeight = permissionSimilarity(row1["App Id"], row2["App Id"])
      if edgeWeight > 0: 
        edge = ((row1["App Id"], row2["App Id"], edgeWeight))
        edgeList.append(edge)
  print(i)
  
with open(path_prefix+'network_edges.txt', 'w') as f: 
  write = csv.writer(f)
  write.writerows(edgeList)

files.download(path_prefix+"network_edges.txt") 

In [5]:
appNet = nx.Graph()
appNet.add_weighted_edges_from(edgeList)

NameError: ignored

In [66]:
import community.community_louvain as community_louvain

communities = community_louvain.best_partition(appNet)

In [67]:
uniq = []
for key in communities:
  if communities[key] not in uniq:
    uniq.append(communities[key])
print(uniq)

[2, 0, 1]


In [68]:
with open(path_prefix+'app_communities_layer1.txt', 'w') as f: 
  f.write(json.dumps(communities))

files.download(path_prefix+"app_communities_layer1.txt") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [108]:
def community_ad_analysis(community):
  ad_free = 0
  ad_supported = 0

  for key in communities:
    if communities[key] == community:
      app = df[df["App Id"]==key]
      isAdSupported = str(list(app["Ad Supported"])[0])
      if isAdSupported == "True":
        ad_supported+=1
      elif isAdSupported == "False":
        ad_free +=1
        
  return ad_free , ad_supported


In [ ]:
def community_free_paid_dist(community):
  free = 0
  paid = 0

  for key in communities:
    if communities[key] == community:
      app = df[df["App Id"]==key]
      isFree= str(list(app["Free"])[0])
      if isFree == "True":
        free+=1
      else:
        paid+=1

  return free, paid


In [ ]:
def community_avg_installs(community):
  total = 0
  count = 0

  for key in communities:
    if communities[key] == community:
      app = df[df["App Id"]==key]
      installs = int(list(app["Maximum Installs"])[0])
      total+= installs
      count+=1

  return total / count

In [111]:
def community_category_analysis(community):
  categories = {}

  for key in communities:
    if communities[key] == community:
      app = df[df["App Id"]==key]
      category = list(app["Category"])[0]
      if category  not in categories:
        categories[category] = 1
      else:
        categories[category] += 1

  return categories

In [ ]:
cat = {}

for comm in uniq:
  cat[comm] = dict()
  cat[comm]["Category Distribution"] = community_category_analysis(comm)
  cat[comm]["AdFree/AdSupported"] = community_ad_analysis(comm)
  cat[comm]["Free/Paid"] = community_ad_analysis(comm)
  cat[comm]["Average Installs"] = community_avg_installs(comm)

In [114]:
import json
with open('layer1.json', 'w') as fp:
    json.dump(cat, fp, indent=4)